In [3]:
#| default_exp updates
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

# Atualização

> Este módulo atualiza as bases. Executa as queries sql do STEL, RADCOM e baixa os arquivos de estações e plano básico do MOSAICO.

In [29]:
#| export
from decimal import Decimal, getcontext
from typing import Union
from urllib.request import urlretrieve, URLError
import xmltodict
from zipfile import ZipFile

import pandas as pd
import pyodbc
from rich.console import Console
from pyarrow import ArrowInvalid, ArrowTypeError
from unidecode import unidecode
from fastcore.xtras import Path
from fastcore.foundation import L
from fastcore.test import test_eq
import pyodbc
from pymongo import MongoClient

from anateldb.constants import *
from anateldb.format import parse_bw, format_types, input_coordenates
from anateldb.functionsdb import ConsultaSRD

getcontext().prec = 5

## Conexão com o banco de dados
A função a seguir é um `wrapper` simples que utiliza o `pyodbc` para se conectar ao banco de dados base da Anatel e retorna o objeto da conexão

In [5]:
#| export
def connect_db(server: str = 'ANATELBDRO05', # Servidor do Banco de Dados
               database: str = 'SITARWEB', # Nome do Banco de Dados
               trusted_conn: str = 'yes', # Conexão Segura: yes | no
               mult_results: bool = True, # Múltiplos Resultados
              )->pyodbc.Connection:
    """Conecta ao Banco `server` e retorna o 'cursor' (iterador) do Banco"""
    return pyodbc.connect(
        "Driver={ODBC Driver 17 for SQL Server};"
        f"Server={server};"
        f"Database={database};"
        f"Trusted_Connection={trusted_conn};"
        f"MultipleActiveResultSets={mult_results};",
        timeout=TIMEOUT,
    )

In [4]:
#echo: false
def test_connection():
    conn = connect_db()
    cursor = conn.cursor()
    for query in (RADCOM,STEL):
        cursor.execute(query)
        test_eq(type(cursor.fetchone()), pyodbc.Row)

In [6]:
#| exporti
def _parse_estações(row: dict)->dict:
    """Given a row in a MongoDB ( a dict of dicts ), it travels some keys and return a subset dict"""
    
    d = {k.replace('@', '').lower():row[k] for k in ("@SiglaServico", "@id", "@state",
        "@entidade",
        "@fistel",
        "@cnpj",
        "@municipio",
        "@uf")}
    entidade = row.get('entidade', {})
    d.update({k.replace('@', '').lower():entidade[k] for k in ('@num_servico', '@habilitacao_DataValFreq')})
    administrativo = row.get('administrativo', {})
    d['numero_estacao'] = administrativo.get('@numero_estacao')
    estacao = row.get('estacao_principal', {})
    d.update({k.replace('@', '').lower():estacao[k] for k in ('@latitude', '@longitude')})
    return d

In [7]:
#| exporti
def _read_estações(path: Union[str, Path]) -> pd.DataFrame:
    """Read the zipped xml file `Estações.zip` from MOSAICO and returns a dataframe"""
    
    with ZipFile(path) as myzip:
        with myzip.open('estacao_rd.xml') as myfile:
            estacoes = xmltodict.parse(myfile.read())
            
    assert 'estacao_rd' in estacoes, "The xml file inside estacoes.zip is not in the expected format"
    assert 'row' in estacoes['estacao_rd'], "The xml file inside estacoes.zip is not in the expected format"
    
    df = pd.DataFrame(L(estacoes['estacao_rd']['row']).map(_parse_estações))
    df = df[df.state.str.contains("-C1$|-C2$|-C3$|-C4$|-C7|-C98$")].reset_index(drop=True)
    df = df.loc[:, COL_ESTACOES]
    df.columns = NEW_ESTACOES    
    for c in df.columns:
        df.loc[df[c] == "", c] = pd.NA
    return df

In [18]:
estacoes_df = _read_estações('../dados/estações.zip')
print(estacoes_df.dtypes)
print(len(estacoes_df.index))
estacoes_df.head()

Serviço                  object
Num_Serviço              object
Status                   object
Entidade                 object
Fistel                   object
UF                       object
Id                       object
Número_Estação           object
Latitude_Transmissor     object
Longitude_Transmissor    object
CNPJ                     object
Validade_RF              object
dtype: object
30163


,Serviço,Num_Serviço,Status,Entidade,Fistel,UF,Id,Número_Estação,Latitude_Transmissor,Longitude_Transmissor,CNPJ,Validade_RF
0,TV,248,TV-C1,REDE DE COMUNICACOES ACREANA LTDA,50442889933,AC,57dbaad04f6cc,<NA>,<NA>,<NA>,01865469000156,<NA>
1,TV,248,TV-C1,X-MEDIAGROUP S.A.,50410887137,AC,57dbaad053c60,<NA>,<NA>,<NA>,03211814000163,<NA>
2,TV,248,TV-C4,TELEVISAO OESTE BAIANO LTDA,06030116240,BA,57dbaad0dc4e3,322647029,-12.101388888889,-44.993611111111,16395923000120,2023-12-31
3,TV,248,TV-C2,TELEVISAO SANTA CRUZ LTDA,06020355110,BA,57dbaad0eb54a,322623553,-14.779444444444,-39.262222222222,13476833000175,2023-12-31
4,TV,248,TV-C4,TV CABRALIA LTDA,06020354903,BA,57dbaad0ef8af,322623537,-14.78167,-39.26167,13494265000135,2023-12-31


In [14]:
#| exporti
def _parse_pb(row: dict)->dict:
    """Given a row in the MongoDB file canais.zip ( a dict of dicts ), it travels some keys and return a subset dict"""
    return {unidecode(k).lower().replace("@", ""): v  for k,v in row.items()}

In [15]:
#| exporti
def _read_plano_basico(path: Union[str, Path]) -> pd.DataFrame:
    """Read the zipped xml file `Plano_Básico.zip` from MOSAICO and returns a dataframe"""    
    df = L()
    with ZipFile(path) as myzip:
        with myzip.open('plano_basicoTVFM.xml') as myfile:
            pbtvfm = xmltodict.parse(myfile.read())
        with myzip.open('plano_basicoAM.xml') as myfile:
            pbam = xmltodict.parse(myfile.read())
        with myzip.open('secundariosTVFM.xml') as myfile:
            stvfm = xmltodict.parse(myfile.read())
        with myzip.open('secundariosAM.xml') as myfile:
            sam = xmltodict.parse(myfile.read())    
            
    for base in (pbtvfm, stvfm, pbam, sam):
        assert 'plano_basico' in base, "The xml files inside canais.zip is not in the expected format"
        assert 'row' in base['plano_basico'], "The xml file inside canais.zip is not in the expected format"
        df.extend(L(base['plano_basico']['row']).map(_parse_pb))
        
    df = pd.DataFrame(df)
    df = df.loc[df.pais == "BRA", COL_PB].reset_index(drop=True)    
    df.columns = NEW_PB
    df = df[df.Status.str.contains("-C1$|-C2$|-C3$|-C4$|-C7|-C98$")].reset_index(drop=True)
    df.loc[:, 'Frequência'] = df.Frequência.str.replace(',', '.')
    for c in df.columns:
        df.loc[df[c] == '', c] = pd.NA
    return df    

In [17]:
plano_basico_df = _read_plano_basico('../dados/canais.zip')
print(plano_basico_df.dtypes)
print(len(plano_basico_df.index))
plano_basico_df.head()

Id                   object
Município            object
Frequência           object
Classe               object
Serviço              object
Entidade             object
Latitude_Estação     object
Longitude_Estação    object
UF                   object
Status               object
CNPJ                 object
Fistel               object
dtype: object
48386


,Id,Município,Frequência,Classe,Serviço,Entidade,Latitude_Estação,Longitude_Estação,UF,Status,CNPJ,Fistel
0,57dbaad04f6cc,Cruzeiro do Sul,207,A,TV,REDE DE COMUNICACOES ACREANA LTDA,"-7,631111111111","-72,67",AC,TV-C1,01865469000156,50442889933
1,57dbaad053c60,Mâncio Lima,539,C,TV,X-MEDIAGROUP S.A.,"-7,6141666666667","-72,895833333333",AC,TV-C1,03211814000163,50410887137
2,57dbaad0dc4e3,Barreiras,79,A,TV,TELEVISAO OESTE BAIANO LTDA,"-12,102222222222","-44,994444444444",BA,TV-C4,16395923000120,06030116240
3,57dbaad0eb54a,Itabuna,69,A,TV,TELEVISAO SANTA CRUZ LTDA,"-14,780555555555","-39,261944444444",BA,TV-C2,13476833000175,06020355110
4,57dbaad0ef8af,Itabuna,177,B,TV,TV CABRALIA LTDA,"-14,78","-39,260833333333",BA,TV-C4,13494265000135,06020354903


In [26]:
#| export
def clean_mosaico(df: pd.DataFrame, # DataFrame com os dados de Estações e Plano_Básico mesclados 
                pasta: Union[str, Path], # Pasta com os dados de municípios para imputar coordenadas ausentes
) -> pd.DataFrame: # DataFrame com os dados mesclados e limpos
    """Clean the merged dataframe with the data from the MOSAICO page"""
    COLS = [c for c in df.columns if "_x" in c]
    for col in COLS:
        col_x = col
        col_y = col.split("_")[0] + "_y"
        df.loc[df[col_x].isna(), col_x] = df.loc[df[col_x].isna(), col_y]
        df.loc[df[col_y].isna(), col_y] = df.loc[df[col_y].isna(), col_x]
        if df[col_x].notna().sum() > df[col_y].notna().sum():
            a, b = col_x, col_y
        else:
            a, b = col_y, col_x
        df.drop(b, axis=1, inplace=True)
        df.rename({a: a[:-2]}, axis=1, inplace=True)

    # df.loc[df.Latitude_Transmissor == "", "Latitude_Transmissor"] = df.loc[
    #     df.Latitude_Transmissor == "", "Latitude_Estação"
    # ]
    # df.loc[df.Longitude_Transmissor == "", "Longitude_Transmissor"] = df.loc[
    #     df.Longitude_Transmissor == "", "Longitude_Estação"
    # ]
    
    # df.loc[df.Latitude_Transmissor.isna(), "Latitude_Transmissor"] = df.loc[
    #     df.Latitude_Transmissor.isna(), "Latitude_Estação"
    # ]
    # df.loc[df.Longitude_Transmissor.isna(), "Longitude_Transmissor"] = df.loc[
    #     df.Longitude_Transmissor.isna(), "Longitude_Estação"
    # ]
    # df.drop(["Latitude_Estação", "Longitude_Estação"], axis=1, inplace=True)
    # df.rename(
    #     columns={
    #         "Latitude_Transmissor": "Latitude",
    #         "Longitude_Transmissor": "Longitude",
    #     },
    #     inplace=True,
    # )

    df = input_coordenates(df, pasta)
    df.loc[:, "Frequência"] = df.Frequência.str.replace(",", ".")    
    df = df[df.Frequência.notna()].reset_index(drop=True)

    # Removido o código abaixo devido a inconsistência no Mosaico
    # if freq_nans := df[df.Frequência.isna()].Id.tolist():
    #     complement_df = scrape_dataframe(freq_nans)
    #     df.loc[
    #         df.Frequência.isna(),
    #         [
    #             "Status",
    #             "Entidade",
    #             "Fistel",
    #             "Frequência",
    #             "Classe",
    #             "Num_Serviço",
    #             "Município",
    #             "UF",
    #         ],
    #     ] = complement_df.values

    df.loc[:, "Frequência"] = df.Frequência.astype("float")
    df.loc[df.Serviço == "OM", "Frequência"] = df.loc[
        df.Serviço == "OM", "Frequência"
    ].apply(lambda x: Decimal(x) / Decimal(1000))
    df.loc[:, "Validade_RF"] = df.Validade_RF.astype("string").str.slice(0, 10)
    return df.drop_duplicates(keep="first").reset_index(drop=True)

In [27]:
mosaico_df = estacoes_df.merge(plano_basico_df, on="Id", how="left")
print(len(mosaico_df.index))
mosaico_df = clean_mosaico(mosaico_df, '../dados')
print(len(mosaico_df.index))
display(mosaico_df.head())
# mosaico_df.to_csv('../dados/mosaico_df_xml.csv', sep=';', encoding='utf-8')

48223
30163


,Num_Serviço,Id,Número_Estação,Latitude,Longitude,Validade_RF,Município,Frequência,Classe,Serviço,Entidade,UF,Status,CNPJ,Fistel,Coordenadas_do_Município
0,248,57dbaad04f6cc,<NA>,-7.631111111111,-72.67,<NA>,Cruzeiro do Sul,207.0,A,TV,REDE DE COMUNICACOES ACREANA LTDA,AC,TV-C1,01865469000156,50442889933,False
1,248,57dbaad053c60,<NA>,-7.6141666666667,-72.895833333333,<NA>,Mâncio Lima,539.0,C,TV,X-MEDIAGROUP S.A.,AC,TV-C1,03211814000163,50410887137,False
2,248,57dbaad0dc4e3,322647029,-12.101388888889,-44.993611111111,2023-12-31,Barreiras,79.0,A,TV,TELEVISAO OESTE BAIANO LTDA,BA,TV-C4,16395923000120,06030116240,False
3,248,57dbaad0eb54a,322623553,-14.779444444444,-39.262222222222,2023-12-31,Itabuna,69.0,A,TV,TELEVISAO SANTA CRUZ LTDA,BA,TV-C2,13476833000175,06020355110,False
4,248,57dbaad0ef8af,322623537,-14.78167,-39.26167,2023-12-31,Itabuna,177.0,B,TV,TV CABRALIA LTDA,BA,TV-C4,13494265000135,06020354903,False


## Atualização das bases de dados
As bases de dados são atualizadas atráves das funções a seguir, o único argumento passado em todas elas é a pasta na qual os arquivos locais processados serão salvos, os nomes dos arquivos são padronizados e não podem ser editados para que as funções de leitura e processamento recebam somente a pasta na qual esses arquivos foram salvos.

In [15]:
#| export
def _save_df(df: pd.DataFrame, folder: Union[str, Path], stem: str) -> pd.DataFrame:
    """Format, Save and return a dataframe"""
    df = format_types(df, stem)
    df = df.drop_duplicates(keep='first').reset_index(drop=True)
    df = df.dropna(subset=['Latitude', 'Longitude']).reset_index(drop=True)
    try:
        file = Path(f"{folder}/{stem}.parquet.gzip")
        df.to_parquet(file, compression="gzip", index=False)
    except (ArrowInvalid, ArrowTypeError):
        file.unlink(missing_ok=True)
        try:
            file = Path(f"{folder}/{stem}.fth")
            df.to_feather(file)
        except (ArrowInvalid, ArrowTypeError):
            file.unlink(missing_ok=True)
            try:
                file = Path(f"{folder}/{stem}.xlsx")
                with pd.ExcelWriter(file) as wb:
                    df.to_excel(
                        wb, sheet_name="DataBase", engine="openpyxl", index=False
                    )
            except Exception as e:
                raise ValueError(f"Could not save {stem} to {file}") from e
    return df



In [11]:
# | export
def update_radcom(
        conn: pyodbc.Connection, # Objeto de conexão de banco
        folder: Union[str, Path] # Pasta onde salvar os arquivos
        
) -> pd.DataFrame: # DataFrame com os dados atualizados
    """Atualiza a tabela local retornada pela query `RADCOM`"""
    console = Console()
    with console.status(
        "[cyan]Lendo o Banco de Dados de Radcom...", spinner="earth"
    ) as status:
        try:            
            df = pd.read_sql_query(RADCOM, conn)
            return _save_df(df, folder, "radcom")
        except pyodbc.OperationalError as e:
            status.console.log(
                "Não foi possível abrir uma conexão com o SQL Server. Esta conexão somente funciona da rede cabeada!"
            )
            raise ConnectionError from e

In [12]:
# | export
def update_stel(
        conn: pyodbc.Connection, # Objeto de conexão de banco
        folder:Union[str, Path] # Pasta onde salvar os arquivos        
) -> pd.DataFrame: # DataFrame com os dados atualizados
    """Atualiza a tabela local retornada pela query `STEL`"""
    console = Console()
    with console.status(
        "[red]Lendo o Banco de Dados do STEL. Processo Lento, aguarde...",
        spinner="bouncingBall",
    ) as status:
        try:            
            df = pd.read_sql_query(STEL, conn)
            return _save_df(df, folder, "stel")
        except pyodbc.OperationalError as e:
            status.console.log(
                "Não foi possível abrir uma conexão com o SQL Server. Esta conexão somente funciona da rede cabeada!"
            )
            raise ConnectionError from e

In [13]:
stel = update_stel(Path.cwd().parent / 'dados')

(  ●   ) Lendo o Banco de Dados do STEL. Processo Lento, aguarde...

In [16]:
stel.to_parquet(Path.cwd().parent / 'dados' / 'stel.parquet.gzip', compression='gzip', index=False)

In [5]:
# | export
def update_mosaico(        
        clientMongoDB: MongoClient, # Ojeto de conexão com o MongoDB
        folder: Union[str, Path] # Pasta onde salvar os arquivos
) -> pd.DataFrame: # DataFrame com os dados atualizados
    """Atualiza a tabela local do Mosaico. É baixado e processado arquivos xml zipados da página pública do Spectrum E"""
    console = Console()
    with console.status(
        "Consolidando os dados do Mosaico...", spinner="clock"
    ) as status:  
        
        # stations, _ = urlretrieve(ESTACOES, f"{folder}/estações.zip")
        # pb, _ = urlretrieve(PLANO_BASICO, f"{folder}/canais.zip")
        # estações = _read_estações(stations)
        # plano_basico = _read_plano_basico(pb)
        df = ConsultaSRD(clientMongoDB)
        df = clean_mosaico(df, folder)
    return _save_df(df, folder, "mosaico")
        



In [23]:
import warnings
warnings.filterwarnings("ignore", message='install "ipywidgets" for Jupyter support')

df = update_mosaico(Path.cwd().parent / 'dados')

In [ ]:
# | export
def update_base(
    conn: pyodbc.Connection, # Objeto de conexão de banco
    clientMongoDB: MongoClient, # Ojeto de conexão com o MongoDB
    folder: Union[str, Path] # Pasta onde salvar os arquivos    
) -> pd.DataFrame: # DataFrame com os dados atualizados
    # sourcery skip: use-fstring-for-concatenation
    """Wrapper que atualiza opcionalmente lê e atualiza as três bases indicadas anteriormente, as combina e salva o arquivo consolidado na folder `folder`"""
    stel = update_stel(conn, folder, ).loc[:, TELECOM]
    radcom = update_radcom(conn, folder).loc[:, SRD]
    mosaico = update_mosaico(clientMongoDB, folder).loc[:, RADIODIFUSAO]    
    radcom["Num_Serviço"] = "231"
    radcom["Status"] = "RADCOM"
    radcom["Classe_Emissão"] = pd.NA
    radcom["Largura_Emissão"] = BW_MAP["231"]
    radcom["Entidade"] = radcom.Entidade.str.rstrip().str.lstrip()
    radcom["Validade_RF"] = pd.NA
    radcom["Fonte"] = "SRD"
    stel["Status"] = "L"
    stel["Entidade"] = stel.Entidade.str.rstrip().str.lstrip()
    stel["Fonte"] = "STEL"
    mosaico["Fonte"] = "MOS"
    mosaico["Classe_Emissão"] = pd.NA
    mosaico["Largura_Emissão"] = mosaico.Num_Serviço.map(BW_MAP)
    rd = (
        pd.concat([mosaico, radcom, stel])
        .sort_values(["Frequência", "Latitude", "Longitude"])
        .reset_index(drop=True)
    )
    rd = rd.drop_duplicates(keep="first").reset_index(drop=True)
    rd["BW(kHz)"] = rd.Largura_Emissão.astype('string').fillna('-1').apply(parse_bw)
    return _save_df(rd, folder, "base")

In [21]:
#| hide
folder = Path.cwd().parent / 'dados'

In [ ]:
# from urllib.request import Request, urlopen
# from urllib.error import URLError
# req = Request(ESTACAO)
# try:
#     response = urlopen(req)
# except URLError as e:
#     if hasattr(e, 'reason'):
#         print('We failed to reach a server.')
#         print('Reason: ', e.reason)
#     elif hasattr(e, 'code'):
#         print('The server couldn\'t fulfill the request.')
#         print('Error code: ', e.code)
# else:
#     Path.cwd().joinpath('estações.zip').write_bytes(response.read())